In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
sys.path.insert(0, '../')
sys.path.append('/home/arya_03/.envs/objdet/lib/python2.7/site-packages/')
import matplotlib
matplotlib.use('Agg')

In [ ]:
from __future__ import division
import os
import numpy as np
import pandas as pd
from skimage.io import imread
import cv2
from pandas import read_csv
import matplotlib.pyplot as plt
import glob

%matplotlib inline

## Visualization

In [ ]:
from evaluate import load_network
import cPickle as pickle
from models import *
from utils import get_file_list, create_fixed_image_shape
from nolearn.lasagne import BatchIterator
from plotting import plot_face_bb
# from lazy_batch_iterator import LazyBatchIterator
import skimage
from sklearn.metrics import confusion_matrix, accuracy_score
from mpl_toolkits.axes_grid1 import AxesGrid

In [ ]:
train_folder = '../train/'
fnames, bboxes = get_file_list(train_folder)

In [ ]:
def prepare_test_img(shape, img=None, name=None):
    from skimage.io import imread 
    im = None
    if img is not None:
        im = img
    elif name is not None:
        im = imread(name)
    inp = create_fixed_image_shape(im, shape, mode='fit')
    return inp

### Option 1 - Load weights

In [ ]:
print NET_CONFIGS

In [ ]:
nnet = load_network("/home/arya_03/work/face_detection_gtx/model_run3_smoothl1_model16_scale_trans_3/model_28.pkl",
                    "config_4c_1234_3d_smoothl1_lr_step",
                    "BatchIterator")

### Option 2 - Full network

### Evaluate result on the test images

In [ ]:
def plot_imgs_grid(Z, ncols=10):
    num = len(Z)
    fig = plt.figure(1, (80, 80))
    fig.subplots_adjust(left=0.05, right=0.95)
    grid = AxesGrid(fig, (1, 4, 2),  # similar to subplot(142)
                    nrows_ncols=(int(np.ceil(float(num) / ncols)), ncols),
                    axes_pad=0.04,
                    share_all=True,
                    label_mode="L",
                    )

    for i in range(num):
        im = grid[i].imshow(Z[i], interpolation="nearest")
    for i in range(grid.ngrids):
        grid[i].axis('off')

    for cax in grid.cbar_axes:
        cax.toggle_label(False)

In [ ]:
# Predict and plot the bounding boxes for images in test_imgs/ folder
outs = []
for fname in glob.glob(os.path.join('../test_imgs', '*.jpg')):
    proc_img = prepare_test_img((256, 256, 3), name=fname)
    img = np.transpose(proc_img, [2, 0, 1])
    preds = nnet.predict([img])
    outs.append(plot_face_bb(proc_img, preds[0], path=False, plot=False))

In [ ]:
# run3_3 model_28
plot_imgs_grid(outs, 5)

In [ ]:
# run3_3 model_1
plot_imgs_grid(outs, 5)

In [ ]:
# run3_2 model_2
plot_imgs_grid(outs, 5)

In [ ]:
# run3 model_25
plot_imgs_grid(outs, 5)

In [ ]:
# run2 model_25
plot_imgs_grid(outs, 5)

In [ ]:
# run3 model_20
plot_imgs_grid(outs, 5)

In [ ]:
# run3 model_19
plot_imgs_grid(outs, 5)

In [ ]:
# run2 model_19
plot_imgs_grid(outs, 5)

In [ ]:
# model_15
plot_imgs_grid(outs, 5)

### Visualize weights

In [ ]:
pp = nnet.get_all_params_values()

In [ ]:
for k in pp.keys():
    if 'conv' in k:
        for w in pp[k]:
            if w.ndim <= 2:
                continue
            for i in range(w.shape[0]):
                plt.figure(figsize=(1,1))
                plt.axis('off')
                plt.imshow(w[i, :, :, :].mean(axis=0), interpolation='nearest')
            break

In [ ]:
def plot_weight_matrix(Z, k):
    num = Z.shape[0]
    fig = plt.figure(1, (30, 30))
    fig.subplots_adjust(left=0.05, right=0.95)
    grid = AxesGrid(fig, (1, 4, 2),  # similar to subplot(142)
                    nrows_ncols=(int(np.ceil(num / 10.)), 10),
                    axes_pad=0.04,
                    share_all=True,
                    label_mode="L",
                    )

    for i in range(num):
        im = grid[i].imshow(Z[i, :, :, :].mean(
            axis=0), cmap='coolwarm')
    for i in range(grid.ngrids):
        grid[i].axis('off')

    for cax in grid.cbar_axes:
        cax.toggle_label(False)

In [ ]:
plot_weight_matrix(pp['conv1_1'][0], 'test')

## Plot x-y-scale for the faces used for training

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
df = read_csv('../aug.csv')
df.head()

In [ ]:
%matplotlib notebook
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(df['r'][:4000], df['c'][:4000], df['width'][:4000], c='none', facecolors='none', edgecolors='r')

In [ ]:
df.describe()

In [ ]:
%matplotlib inline
plt.hist2d(df['r'], df['c'], bins=30);